## TEST Code for running and evaluating Steadiness / Cohesiveness and other metrics

#### Import packages & set dataset path

In [4]:
from helper import *

# import helper as h
import sys

## Import Metrics
sys.path.append("/home/hj/vis2021_snc/metrics4mdp/metrics4mdp")
from provider import MDPMetricProvider



PATH_TO_DATASET = "../../dataset-generator/dataset/"

#### Pilot Dataset Load

In [7]:
mnist_umap_path = make_path(PATH_TO_DATASET, "mnist", "umap", 4, [0.5, 10])
emb_data, raw_data, label_data = get_data(mnist_umap_path)

print(emb_data[:4])

[[-3.4933080673217773, 5.042341709136963], [-1.5574856996536255, -0.05618373304605484], [4.868945121765137, 3.878509759902954], [-1.8701975345611572, 2.784097909927368]]


#### Pilot Local Metrics Usage

In [9]:
mlist = ["Sammon", "DTM", "DTM_KL1", "Trustworthiness", "Continuity", "MRRE_XZ", "MRRE_ZX"]
metric_provider = MDPMetricProvider(raw_data, emb_data, mlist, 7) 
result = metric_provider.run()

In [10]:
print(result)

{'Sammon': 0.994978622080915, 'DTM': 0.45324987, 'DTM_KL1': 0.002034952, 'Trustworthiness': 0.9435867761005567, 'Continuity': 0.9737871089938587, 'MRRE_XZ': 0.9795523391209287, 'MRRE_ZX': 0.9440351635881546}
